In [107]:
import getpass
from langchain import HuggingFaceHub
import os
from dotenv import load_dotenv

from pathlib import Path
dotenv_path = Path('D:\Coding\langchain gen ai\langchain_projects\Langchain_projects\.env')
load_dotenv(dotenv_path=dotenv_path)

GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')
ANTHROPIC_API_KEY = os.getenv('Anthropic_API_KEY')
GROQ_API_KEY = os.getenv('groq_api_key')

In [15]:
# if "GOOGLE_API_KEY" not in os.environ:
#     os.environ["GOOGLE_API_KEY"] = getpass.getpass("Provide your Google API Key")
# if "HUGGINGFACEHUB_API_TOKEN" not in os.environ:
#     os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass.getpass("Provide your HUGGINGFACE_API_TOKEN")

In [49]:
llm_hf = HuggingFaceHub(repo_id = "google/flan-t5-large", model_kwargs = {"temperature": 2, "max_length": 50})

In [50]:
output= llm_hf.invoke("can you give me sentences that end with apple")
print(output)

Sentence: Then he sat down and ate a piece of apple. Then he sat down and ate a piece of apple. Then he sat down and ate


# Promt templates


In [51]:
from langchain.prompts import PromptTemplate

In [52]:
prompt = PromptTemplate(input_variables = ["country"], 
                                 template = "Tell me the capital of {country}")

In [55]:
chain = prompt | llm_hf
chain.invoke("USA")

'detroit'

### Combining multiple chains usig seq chains

In [ ]:
from langchain.schema.runnable import RunnablePassthrough

In [63]:
capital_prompt = PromptTemplate(input_variables=["country"],
                             template = "Please provide capital of the {country}")
adventure_prompt = PromptTemplate(input_variables=["capital"],
                             template = "Please provide some adventurous places of the {capital}")
capital_chain = capital_prompt | llm_hf
adventure_chain = adventure_prompt | llm_hf

chain = ({"capital" : capital_chain} 
        | RunnablePassthrough.assign(optimization=adventure_chain))

result = chain.invoke({"india"})

In [81]:
capital_prompt = PromptTemplate(input_variables=["country"],
                             template = "Please provide capital of the {country}")
famous_prompt = PromptTemplate(input_variables=["capital"],
                             template = "write 5 famous places in {capital}")
capital_chain = capital_prompt | llm_hf
famous_chain = adventure_prompt | llm_hf
chain = ({"capital" : capital_chain} 
        | RunnablePassthrough.assign(review=famous_chain))

result = chain.invoke({"country": 'India'})
result

{'capital': 'chennai', 'review': 'Theme Parks'}

## Sequential chains with LCEL


In [85]:
combined_chain = capital_chain | famous_chain
combined_chain.invoke({'country':'India'})

'Theme Parks'

# chatmodels with Anthropic

In [94]:
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain_anthropic import ChatAnthropic

In [101]:
chatllm = ChatAnthropic(
    model="claude-3-sonnet-20240229",
    # model='claude-2.1',
    temperature=0,
    max_tokens=110,
    timeout=None,
    # max_retries=2,
    # other params...
)

In [ ]:
chatllm([
    SystemMessage(content="You are a Life Science expert"),
    HumanMessage(content = "Please provide me some expert opinion on cancer")
])

# chatmodels with Groq

In [117]:
from langchain_groq import ChatGroq
from langchain.schema import HumanMessage, SystemMessage, AIMessage

In [120]:
chat_groq = ChatGroq(
    temperature=0,
    model="llama3-70b-8192",
    # api_key="" # Optional if not set as an environment variable
)

In [121]:
chat_groq([
    SystemMessage(content="You are a Life Science expert"),
    HumanMessage(content = "Please provide me some expert opinion on cancer")

])

AIMessage(content="As a Life Science expert, I'd be happy to provide some insights on cancer.\n\n**What is Cancer?**\n\nCancer is a complex and multifaceted disease characterized by the uncontrolled growth and spread of abnormal cells. It can affect any part of the body and is caused by a combination of genetic and environmental factors.\n\n**Types of Cancer**\n\nThere are over 100 different types of cancer, each with its unique characteristics and behaviors. Some of the most common types of cancer include:\n\n1. Carcinomas: These are the most common type of cancer and arise from epithelial cells, which are the cells that line the surfaces of organs and glands. Examples include breast cancer, lung cancer, and colon cancer.\n2. Sarcomas: These are cancers that arise from connective tissue cells, such as bone, cartilage, and fat. Examples include osteosarcoma (bone cancer) and leiomyosarcoma (smooth muscle cancer).\n3. Leukemias: These are cancers of the blood and bone marrow. Examples i

## Chat Prompt template with Groq

In [122]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

In [ ]:
system = "You are a Life Science expert."
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | chat
chain.invoke({"text": "Please provide me some expert opinion on cancer."})

# Chat with Ouput parsers

In [1]:
from langchain_groq import ChatGroq
from langchain.schema import BaseOutputParser
from langchain_core.prompts import ChatPromptTemplate


In [2]:
class commaseperatedoutput(BaseOutputParser):
    def parse(self, text:str):
        return text.strip().split(",")

In [10]:
system_template = "You are a healthcare expert. When the user gives any input disease you only provide 5 related diseases seperated by comma ,"
human_template = "{text}"
chatprompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", human_template)
])
chatprompt

ChatPromptTemplate(input_variables=['text'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a healthcare expert. When the user gives any input disease you only provide 5 related diseases seperated by comma ,')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='{text}'))])

In [11]:
chat_groq = ChatGroq(
    temperature=0,
    model="llama3-70b-8192",
    # api_key="" # Optional if not set as an environment variable
)

In [12]:
chat_chain = chatprompt | chat_groq | commaseperatedoutput()

In [13]:
chat_chain

ChatPromptTemplate(input_variables=['text'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a healthcare expert. When the user gives any input disease you only provide 5 related diseases seperated by comma ,')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='{text}'))])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000197F0364940>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000197F03654B0>, model_name='llama3-70b-8192', temperature=1e-08, groq_api_key=SecretStr('**********'))
| commaseperatedoutput()

In [14]:
chat_chain.invoke({"text": "cancer"})

['Here are 5 related diseases to cancer:\n\nLeukemia',
 ' Lymphoma',
 ' Sarcoma',
 ' Melanoma',
 ' Carcinoma']